# SIMD: El paralelismo que puede (a veces) ocurrir automáticamente

SIMD: Single-Intruction, Multiple Data

(También llamado confusamente vectorización)

## The architecture

En lugar de calcular cuatro sumas secuencialmente:

\begin{align}
x_1 + y_1 &\rightarrow z_1 \\
x_2 + y_2 &\rightarrow z_2 \\
x_3 + y_3 &\rightarrow z_3 \\
x_4 + y_4 &\rightarrow z_4
\end{align}

Los procesadores modernos tienen ununidades de procesamiento de vectores que pueden hacerlo todo a la vez:

$$
\left(\begin{array}{cc}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}\right)
+
\left(\begin{array}{cc}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right)
\rightarrow
\left(\begin{array}{cc}
z_1 \\
z_2 \\
z_3 \\
z_4
\end{array}\right)
$$

## Hacer que suceda

Tarea sencilla: calcular la suma de un vector:

In [2]:
A = rand(100_000)
function simplesum(A)
    result = zero(eltype(A))
    for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end

simplesum(A)

49943.29477922153

In [3]:
using BenchmarkTools
@btime simplesum($A)

  117.900 μs (0 allocations: 0 bytes)


49943.29477922153

Entonces, ¿eso es bueno?

In [4]:
@btime sum($A)

  9.700 μs (0 allocations: 0 bytes)


49943.29477922161

Somos más lentos que la suma integrada, ¡y también obtenemos una respuesta diferente! Veamos qué sucede con un flotante de 32 bits en lugar de uno de 64 bits. Cada elemento tiene la mitad del número de bits, así que también dupliquemos la longitud (para que el número total de bits procesados ​​permanezca constante).

In [5]:
A32 = rand(Float32, length(A)*2)
@btime simplesum($A32)
@btime sum($A32);

  190.900 μs (0 allocations: 0 bytes)
  9.200 μs (0 allocations: 0 bytes)


Miremos ¡Eso es aún peor! ¿Que está pasando aqui? Estamos viendo un número par múltiplo diferencia en nuestro rendimiento: ¿quizás la suma integrada de Julia está utilizando algún paralelismo? Intentemos usar SIMD nosotros mismos: 

In [6]:
function simdsum(A)
    result = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end
@btime simdsum($A)
@btime simdsum($A32)

  6.120 μs (0 allocations: 0 bytes)
  6.120 μs (0 allocations: 0 bytes)


99960.83f0

¿Qué hizo eso y por qué no siempre usamos `@simd`, o por qué Julia no siempre usa `@simd` para cada bucle for automáticamente? Mira los valores:

In [7]:
simplesum(A), simdsum(A), sum(A)

(49943.29477922153, 49943.2947792216, 49943.29477922161)

In [8]:
simplesum(A32), simdsum(A32), sum(A32)

(99960.2f0, 99960.83f0, 99960.82f0)

Without `@simd`, Julia is doing _exactly_ what we told it to do: it's taking
each element of our array and adding it to a big pile sequentially. Our answer
is smaller than what Julia's builtin `sum` thinks it is: that's because as our
pile gets bigger we begin losing the lower bits of each element that we're
adding, and those small losses begin to add up!

The `@simd` macro tells Julia that it can re-arrange floating point additions —
even if it would change the answer. Depending on your CPU, this may lead to 2x or 4x
or even 8x parallelism. Essentially, Julia is computing independent sums for
the even indices and the odd indices simultaneously:


¿Por qué no son iguales?

Sin `@simd`, Julia está haciendo _exactamente_ lo que le dijimos que hiciera: está tomando cada elemento de nuestra matriz y agregándolo a una gran pila secuencialmente. Nuestra respuesta es más pequeña de lo que piensa la suma integrada de Julia `sum`: eso se debe a que a medida que nuestra pila crece, comenzamos a perder las partes inferiores de cada elemento que estamos agregando, ¡y esas pequeñas pérdidas comienzan a acumularse!

La macro `@simd` le dice a Julia que puede reorganizar las adiciones de punto flotante, incluso si cambiaría la respuesta. Dependiendo de su CPU, esto puede conducir a un paralelismo de 2x, 4x o incluso 8x. Esencialmente, Julia está calculando sumas independientes para los índices pares y los índices impares simultáneamente:

\begin{align}
impares &\leftarrow 0 \\
pares &\leftarrow 0 \\
\text{loop}&\ \text{impar}\ i: \\
    &\left(\begin{array}{cc}
impares \\
pares
\end{array}\right)
\leftarrow
\left(\begin{array}{cc}
impares \\
pares
\end{array}\right)
+
\left(\begin{array}{cc}
x_{i} \\
x_{i+1}
\end{array}\right) \\
total &\leftarrow pares + impares
\end{align}

En muchos casos, Julia puede y sabe que un bucle `for` puede ser " *vectorizado* " (`SIMD-ed`) y se aprovechará de esto de forma predeterminada.

In [9]:
B = rand(1:10, 100_000)
@btime simplesum($B)
@btime sum($B)
B32 = rand(Int32(1):Int32(10), length(B)*2)
@btime simplesum($B32)
@btime simdsum($B32)

  6.060 μs (0 allocations: 0 bytes)
  7.600 μs (0 allocations: 0 bytes)
  6.120 μs (0 allocations: 0 bytes)
  6.120 μs (0 allocations: 0 bytes)


1098730

¿Cómo podemos ver si algo se está vectorizando?

In [12]:
@code_llvm simdsum(A32)

;  @ In[6]:1 within `simdsum`
; Function Attrs: uwtable
define float @julia_simdsum_1906({}* nonnull align 16 dereferenceable(40) %0) #0 {
top:
;  @ In[6]:3 within `simdsum`
; ┌ @ simdloop.jl:69 within `macro expansion`
; │┌ @ abstractarray.jl:279 within `eachindex`
; ││┌ @ abstractarray.jl:116 within `axes1`
; │││┌ @ abstractarray.jl:95 within `axes`
; ││││┌ @ array.jl:151 within `size`
       %1 = bitcast {}* %0 to {}**
       %2 = getelementptr inbounds {}*, {}** %1, i64 3
       %3 = bitcast {}** %2 to i64*
       %4 = load i64, i64* %3, align 8
; └└└└└
; ┌ @ simdloop.jl:72 within `macro expansion`
; │┌ @ int.jl:83 within `<`
    %.not = icmp eq i64 %4, 0
; │└
   br i1 %.not, label %L30, label %L13.lr.ph

L13.lr.ph:                                        ; preds = %top
   %5 = bitcast {}* %0 to float**
   %6 = load float*, float** %5, align 8
; └
; ┌ @ simdloop.jl:75 within `macro expansion`
   %min.iters.check = icmp ult i64 %4, 32
   br i1 %min.iters.check, label %L13, label %vec

    %22 = load float, float* %21, align 4
; │└
; │┌ @ float.jl:398 within `+`
    %23 = fadd fast float %value_phi7, %22
; └└
; ┌ @ simdloop.jl:78 within `macro expansion`
; │┌ @ int.jl:87 within `+`
    %24 = add nuw nsw i64 %value_phi18, 1
; └└
; ┌ @ simdloop.jl:75 within `macro expansion`
; │┌ @ int.jl:83 within `<`
    %exitcond.not = icmp eq i64 %24, %4
; │└
   br i1 %exitcond.not, label %L30, label %L13

L30:                                              ; preds = %L13, %middle.block, %top
   %value_phi2 = phi float [ 0.000000e+00, %top ], [ %20, %middle.block ], [ %23, %L13 ]
; └
;  @ In[6]:6 within `simdsum`
  ret float %value_phi2
}


Entonces, ¿cuáles son los desafíos?
* El mayor obstáculo es que debe convencer a Julia y LLVM de que puede usar instrucciones SIMD para su algoritmo dado. Eso no siempre es posible.
* Hay muchas limitaciones de lo que puede y no puede ser vectorizado (*SIMD-ed*):

In [13]:
@doc @simd

```
@simd
```

Annotate a `for` loop to allow the compiler to take extra liberties to allow loop re-ordering

!!! warning
    This feature is experimental and could change or disappear in future versions of Julia. Incorrect use of the `@simd` macro may cause unexpected results.


The object iterated over in a `@simd for` loop should be a one-dimensional range. By using `@simd`, you are asserting several properties of the loop:

  * It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
  * Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In many cases, Julia is able to automatically vectorize inner for loops without the use of `@simd`. Using `@simd` gives the compiler a little extra leeway to make it possible in more situations. In either case, your inner loop should have the following properties to allow vectorization:

  * The loop must be an innermost loop
  * The loop body must be straight-line code. Therefore, [`@inbounds`](@ref) is   currently needed for all array accesses. The compiler can sometimes turn   short `&&`, `||`, and `?:` expressions into straight-line code if it is safe   to evaluate all operands unconditionally. Consider using the [`ifelse`](@ref)   function instead of `?:` in the loop if it is safe to do so.
  * Accesses must have a stride pattern and cannot be "gathers" (random-index   reads) or "scatters" (random-index writes).
  * The stride should be unit stride.

!!! note
    The `@simd` does not assert by default that the loop is completely free of loop-carried memory dependencies, which is an assumption that can easily be violated in generic code. If you are writing non-generic code, you can use `@simd ivdep for ... end` to also assert that:


  * There exists no loop-carried memory dependencies
  * No iteration ever waits on a previous iteration to make forward progress.


* Debe pensar en las consecuencias de reordenar su algoritmo.

## Un caso un poco más complicado

In [14]:
using BenchmarkTools

In [15]:
function diff!(A, B)
    A[1] = B[1]
    for i in 2:length(A)
        @inbounds A[i] = B[i] - B[i-1]
    end
    return A
end
A = zeros(Float32, 100_000)
B = rand(Float32, 100_000)

diff!(A, B)
[B[1];diff(B)] == A

true

In [16]:
@btime diff!($A, $B)
@btime diff($B);

  8.700 μs (0 allocations: 0 bytes)
  24.500 μs (2 allocations: 390.67 KiB)


¿Pero qué pasa si en su lugar hacemos?

In [17]:
Bcopy = copy(B)
@btime diff!($Bcopy, $Bcopy);

  214.800 μs (0 allocations: 0 bytes)


¿Que sucedió?

In [18]:
@code_llvm diff!(A, B)

;  @ In[15]:1 within `diff!`
; Function Attrs: uwtable
define nonnull {}* @"japi1_diff!_2275"({}* %0, {}** %1, i32 %2) #0 {
top:
  %3 = alloca {}**, align 8
  store volatile {}** %1, {}*** %3, align 8
  %4 = load {}*, {}** %1, align 8
  %5 = getelementptr inbounds {}*, {}** %1, i64 1
  %6 = load {}*, {}** %5, align 8
;  @ In[15]:2 within `diff!`
; ┌ @ array.jl:861 within `getindex`
   %7 = bitcast {}* %6 to { i8*, i64, i16, i16, i32 }*
   %8 = getelementptr inbounds { i8*, i64, i16, i16, i32 }, { i8*, i64, i16, i16, i32 }* %7, i64 0, i32 1
   %9 = load i64, i64* %8, align 8
   %.not = icmp eq i64 %9, 0
   br i1 %.not, label %oob, label %idxend

L16:                                              ; preds = %middle.block, %vector.memcheck, %L16.preheader, %L16
   %value_phi5 = phi i64 [ %18, %L16 ], [ %ind.end, %middle.block ], [ 2, %L16.preheader ], [ 2, %vector.memcheck ]
; └
;  @ In[15]:4 within `diff!`
; ┌ @ array.jl:861 within `getindex`
   %10 = add nsw i64 %value_phi5, -1
   %11 = g

  %scevgep = getelementptr float, float* %28, i64 1
  %scevgep13 = getelementptr float, float* %28, i64 %22
  %scevgep15 = getelementptr float, float* %25, i64 %22
  %bound0 = icmp ult float* %scevgep, %scevgep15
  %bound1 = icmp ult float* %25, %scevgep13
  %found.conflict = and i1 %bound0, %bound1
  br i1 %found.conflict, label %L16, label %vector.ph

vector.ph:                                        ; preds = %vector.memcheck
  %n.vec = and i64 %30, -32
  %ind.end = or i64 %n.vec, 2
  br label %vector.body

vector.body:                                      ; preds = %vector.body, %vector.ph
  %index = phi i64 [ 0, %vector.ph ], [ %index.next, %vector.body ]
  %offset.idx = or i64 %index, 2
; ┌ @ array.jl:861 within `getindex`
   %31 = add nsw i64 %offset.idx, -1
   %32 = getelementptr inbounds float, float* %25, i64 %31
   %33 = bitcast float* %32 to <8 x float>*
   %wide.load = load <8 x float>, <8 x float>* %33, align 4
   %34 = getelementptr inbounds float, float* %32, i64 8
   %

   store <8 x float> %51, <8 x float>* %59, align 4
   %index.next = add i64 %index, 32
   %60 = icmp eq i64 %index.next, %n.vec
   br i1 %60, label %middle.block, label %vector.body

middle.block:                                     ; preds = %vector.body
; └
  %cmp.n = icmp eq i64 %30, %n.vec
  br i1 %cmp.n, label %L35, label %L16
}


Podemos asegurar manualmente que las matrices no tengan solapamiento (o tengan alguna dependencia de bucle), con el muy especial indicador `@simd ivdep` , pero esto puede ser desastroso:

In [19]:
function unsafe_diff!(A, B)
    A[1] = B[1]
    @simd ivdep for i in 2:length(A)
        @inbounds A[i] = B[i] - B[i-1]
    end
    return A
end
@btime unsafe_diff!($A, $B)
[B[1];diff(B)] == A
Bcopy = copy(B)
unsafe_diff!(Bcopy, Bcopy)
[B[1];diff(B)] == Bcopy

  8.500 μs (0 allocations: 0 bytes)


false

Si realmente quieres ensuciarte las manos, puede usar el paquete [SIMD.jl](https://github.com/eschnett/SIMD.jl) para especificar manualmente esas cosas `<8 x float>` que genera LLVM. PERO: esto es complicado y doloroso; a menudo es solo para estar consciente de lo que hace que el código de Julia sea automáticamente *SIMD-able* o vectorizable, algunos de los casos en los que puede fallar y cómo verificar su funcionamiento.

## SIMD

* Explota el paralelismo incorporado en un procesador
* Los mejores bucles `for` son más internos pequeños y ajustados
* A menudo sucede automáticamente si eres cuidadoso.
    * Sigue los consejos de [performance best practices](https://docs.julialang.org/en/v1/manual/performance-tips/)
    *`@inbounds` accede a cualquier matriz 
    * Sin bifurcaciones o llamadas a funciones (no alineadas)
* Puede usar `@simd` para permitir que Julia rompa algunas reglas para que esto suceda
    * ¡Pero tenga cuidado, especialmente con `@simd ivdep`!
* Dependiendo del procesador y los tipos involucrados, puede generar ganancias de 2 a 16 veces con una sobrecarga extraordinariamente pequeña
    * Los tipos de datos más pequeños pueden mejorar esto aún más; use Float32 en lugar de Float64 si es posible, Int32 en lugar de Int64, etc.
    * Al comprar un nuevo procesador, busque soporte [AVX-512](https://en.wikichip.org/wiki/x86/avx-512)